In [1]:
pip install kaggle

UPLOAD KAGGLE JASON FILE

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


**UPLOAD KAGGLE DATA SET**

In [3]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 94% 76.0M/80.9M [00:00<00:00, 120MB/s]
100% 80.9M/80.9M [00:00<00:00, 123MB/s]


In [4]:
! mkdir sentiment140

! unzip sentiment140.zip -d sentiment140

Archive:  sentiment140.zip
  inflating: sentiment140/training.1600000.processed.noemoticon.csv  


In [5]:
from zipfile import ZipFile
dataset = "/content/sentiment140.zip"

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("The DataSet is extracted")


The DataSet is extracted


In [6]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [7]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [9]:
twitter_data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv", encoding='ISO-8859-1')

In [10]:
twitter_data.shape

(1599999, 6)

In [11]:
twitter_data.head(10)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
5,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
6,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
7,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
8,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?
9,0,1467812416,Mon Apr 06 22:20:16 PDT 2009,NO_QUERY,erinx3leannexo,spring break in plain city... it's snowing


In [12]:
column_names = ['target','id','date','flag','user','text']
twitter_data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv", names=column_names, encoding='ISO-8859-1')

In [13]:
twitter_data.shape

(1600000, 6)

In [15]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [16]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [17]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [18]:
twitter_data.replace({'target':{4:1}} , inplace=True)

In [19]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [20]:
port_steam = PorterStemmer()

In [21]:
def stemming(content):
  stemmed_content = re.sub('[a-zA-z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_steam.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [22]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","@ :// . /2 1 - , ' . . ;"
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,' ... . !
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,@ . 50%
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","@ , ' . ' . ? ' ."


In [23]:
print(twitter_data['stemmed_content'] )

0          @ :// . /2 1 - , ' . . ;
1                         ' ... . !
2                           @ . 50%
3                                  
4                 @ , ' . ' . ? ' .
                     ...           
1599995                           .
1599996         . - ! â« :// . /~8
1599997                           ?
1599998                      38 !!!
1599999                   # @ @ @ 4
Name: stemmed_content, Length: 1600000, dtype: object


In [24]:
print(twitter_data['target'] )

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [25]:
x = twitter_data['stemmed_content'].values
y = twitter_data['target'].values

In [26]:
print(x)

["@ :// . /2 1 - , ' . . ;" "' ... . !" '@ . 50%' ... '?' '38 !!!'
 '# @ @ @ 4']


In [27]:
print(y)

[0 0 0 ... 1 1 1]


In [28]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2)

In [29]:
print(x.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


In [30]:
print(x_train)

['' "@ ' !" '' ... '' "! ' !" '. .']


In [31]:
print(x_test)

["@ - ! ' . & ; ." '/ & ;' '@ !' ... '@ 41 ! 4700 !' '' '@ !!!! !']


In [32]:
vectorizer = TfidfVectorizer()

x_train = vectorizer.fit_transform(x_train.tolist())
x_test = vectorizer.transform(x_test)


In [33]:
print(x_train)

  (5, 3436)	1.0
  (6, 7912)	0.7382151603694854
  (6, 7872)	0.6745653244872989
  (20, 2302)	1.0
  (30, 530)	1.0
  (45, 1436)	1.0
  (46, 733)	1.0
  (61, 732)	1.0
  (74, 5735)	1.0
  (88, 6225)	1.0
  (96, 4193)	0.5987608728860242
  (96, 4455)	0.8009278476247198
  (99, 4015)	1.0
  (106, 6970)	1.0
  (107, 2508)	1.0
  (108, 4275)	1.0
  (109, 1)	0.6527945207385148
  (109, 1774)	0.75753502473072
  (110, 6718)	1.0
  (123, 4779)	0.5532665234794572
  (123, 4728)	0.8330042941047515
  (136, 732)	1.0
  (139, 5596)	1.0
  (142, 6075)	1.0
  (149, 3881)	0.6442386391419473
  :	:
  (1279817, 6864)	1.0
  (1279826, 6864)	1.0
  (1279836, 1436)	1.0
  (1279842, 5890)	1.0
  (1279843, 5016)	1.0
  (1279848, 778)	1.0
  (1279849, 3106)	1.0
  (1279851, 3514)	1.0
  (1279861, 1647)	1.0
  (1279863, 4431)	1.0
  (1279864, 2655)	1.0
  (1279880, 2656)	1.0
  (1279894, 300)	0.8048779218451179
  (1279894, 1193)	0.593440419019706
  (1279900, 4492)	1.0
  (1279913, 5503)	1.0
  (1279914, 3014)	1.0
  (1279917, 732)	1.0
  (1279934, 

In [34]:
print(x_test)

  (6, 6971)	1.0
  (15, 2453)	1.0
  (19, 1588)	1.0
  (20, 6614)	1.0
  (23, 3516)	1.0
  (37, 2453)	1.0
  (41, 6913)	1.0
  (54, 892)	1.0
  (61, 6022)	1.0
  (64, 732)	1.0
  (66, 3576)	0.718939351866453
  (66, 2508)	0.45466494082953657
  (66, 375)	0.5257432832840746
  (78, 1436)	1.0
  (92, 6587)	1.0
  (110, 7698)	1.0
  (113, 963)	1.0
  (114, 4015)	1.0
  (123, 3514)	1.0
  (131, 7698)	1.0
  (149, 4609)	1.0
  (170, 733)	1.0
  (176, 733)	1.0
  (184, 732)	1.0
  (195, 6718)	1.0
  :	:
  (319755, 2479)	1.0
  (319767, 5158)	1.0
  (319773, 657)	1.0
  (319834, 3514)	1.0
  (319844, 2479)	1.0
  (319849, 6772)	1.0
  (319860, 2479)	0.6699066273367025
  (319860, 0)	0.7424453586967625
  (319862, 6225)	1.0
  (319878, 4192)	0.7761465030729781
  (319878, 3514)	0.6305526193487644
  (319879, 6311)	1.0
  (319887, 732)	1.0
  (319898, 5898)	1.0
  (319907, 3275)	1.0
  (319917, 3436)	1.0
  (319922, 5344)	1.0
  (319929, 815)	1.0
  (319952, 1793)	1.0
  (319961, 6510)	1.0
  (319970, 1436)	1.0
  (319982, 3104)	1.0
  (319

In [35]:
 model = LogisticRegression(max_iter=1000)


In [36]:
model.fit(x_train , y_train)

LogisticRegression(max_iter=1000)

In [37]:
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(y_train,x_train_prediction)

In [38]:
print("ACCURACY OF THE MODEL IS:", training_data_accuracy)

ACCURACY OF THE MODEL IS: 0.51553984375


In [39]:
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(y_test,x_test_prediction)

In [40]:
print("accuracy on training data is:", test_data_accuracy )

accuracy on training data is: 0.51070625


In [41]:
import pickle

In [42]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [43]:
loaded_model = pickle.load(open('trained_model.sav','rb'))



In [44]:
x_new = x_test[200]
print(y_test[200])

prediction = model.predict(x_new)
print(prediction)

if (prediction[0]==0):
  print('negative')
else:
  print('positive')

1
[0]
negative
